In [ ]:
!python3.11 -m pip install --upgrade pip

In [ ]:
!python3 -m pip install llama-index llama-cpp-python fpdf
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

In [ ]:
!pip3 install numpy

In [ ]:
import numpy as np

arr = np.array([1, 2, 3])

print(arr)

In [1]:

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

# from fpdf import FPDF

In [2]:
model_url = 'llama-2-7b-chat.Q3_K_M.gguf'

In [4]:
# LOAD MODEL 
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_path=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    # model_url=None,
    temperature=0.1,
    max_new_tokens=512,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    # context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    # model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

# llm = LlamaCPP(
#         model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q3_K_M.gguf",
#         model_path=None,
#         temperature=0,
#         max_new_tokens=512,
#         context_window=10000,
#         generate_kwargs={},
#         verbose=True,
#     )


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

In [14]:
prompt ='''
Rewrite the user's accomplishment for a resume in the following format: I accomplished X by the measure Y that resulted in Z. Short and concise sentences.

accomplishment: {}
'''


In [19]:
accomplishment=input("what did u achieve")

In [20]:
final_question = prompt.format(accomplishment)
print(final_question)



Rewrite the user's accomplishment for a resume in the following format: I accomplished X by the measure Y that resulted in Z. Short and concise sentences.

accomplishment: 4x as many ways to access data by implementing a system to retrieve and store information from a variety of sources (REST API, SharePoint/local excel & csvs), resulting in a unified data storage in PostgreSQL



In [21]:
response_iter = llm.stream_complete(final_question)
str_res = ""
for response in response_iter:
    str_res += response.delta
    print(response.delta, end="", flush=True)

Llama.generate: prefix-match hit


  Sure! Here is the rewritten accomplishment in the requested format:
I accomplished increasing data accessibility by implementing a system to retrieve and store information from multiple sources (REST API, SharePoint/local excel & csvs) resulting in a unified data storage in PostgreSQL, achieving 4x more ways to access data.


llama_print_timings:        load time =    3837.95 ms
llama_print_timings:      sample time =       7.44 ms /    72 runs   (    0.10 ms per token,  9674.82 tokens per second)
llama_print_timings: prompt eval time =    1848.50 ms /    52 tokens (   35.55 ms per token,    28.13 tokens per second)
llama_print_timings:        eval time =    5261.54 ms /    71 runs   (   74.11 ms per token,    13.49 tokens per second)
llama_print_timings:       total time =    7172.84 ms /   123 tokens


In [ ]:
import textwrap

def text_to_pdf(text, filename):
    a4_width_mm = 210
    pt_to_mm = 0.35
    fontsize_pt = 10
    fontsize_mm = fontsize_pt * pt_to_mm
    margin_bottom_mm = 10
    character_width_mm = 7 * pt_to_mm
    width_text = a4_width_mm / character_width_mm

    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.set_auto_page_break(True, margin=margin_bottom_mm)
    pdf.add_page()
    pdf.set_font(family='Arial', size=fontsize_pt)
    splitted = text.split('\n')

    for line in splitted:
        lines = textwrap.wrap(line, width_text)

        if len(lines) == 0:
            pdf.ln()

        for wrap in lines:
            pdf.cell(0, fontsize_mm, wrap, ln=1)

    pdf.output(filename, 'F')

In [ ]:
print(str_res)
# save FPDF() class into a 
# variable pdf
pdf = FPDF()

text_to_pdf(str_res, "resume.pdf")